In [1]:
# Hand-off Notes
# This script was mainly written by David, I've added to it and added in come extra comments

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn import preprocessing

# Data used for training
f = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_1\2_Types_TrainingData_edited.csv' 
f2 = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_1\2_Types_TrainingData_NoFlats.csv'
f3 = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_1\All_RoofTypes.csv'
data = pd.read_csv(f3)
data.head()

# The raw data we will be predicting house_types for
stage1_f = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_1\Roof_Application_Set_Small.csv'
stage1_data = pd.read_csv(stage1_f)

In [2]:
#listing all the variables in the training data to use for the prediction
features = ['bb_lngTOshrt_ratio','bbTOfp_ratio','ratio_3lngTO3hrt','fp_seg_num','fp_avg_seg_len_3long','fp_avg_seg_len_3short','seg_numTOunique_lengths']
X = data[features]
Y = data['house_type']
stage1_X = stage1_data[features]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)
print(len(X_train), 'train examples')
print(len(X_test), 'test examples')


(184, 'train examples')
(80, 'test examples')


In [3]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
stage1_X = sc.transform(stage1_X)

C:\Users\jrdnb\Anaconda2.0\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\jrdnb\Anaconda2.0\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\jrdnb\Anaconda2.0\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
C:\Users\jrdnb\Anaconda2.0\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [4]:
from sklearn.svm import SVC
classifier1=SVC(kernel ='rbf', C=1, gamma=1)
classifier2=SVC(kernel ='linear', gamma=1)
classifier1.fit(X_train, Y_train)
classifier2.fit(X_train, Y_train)

y_pred1= classifier1.predict(X_test)
y_pred2= classifier2.predict(X_test)
#print X_train
#print y_pred1
#c1 = pd.DataFrame({'house_type': y_pred1, 'house_id': X_train['house_id']}).to_csv('classification1.csv')
#c2 = pd.DataFrame(y_pred2, columns=['house_type']).to_csv('classification2.csv')

from sklearn.metrics import accuracy_score
print('Linear SVM Prediction Accuracy = ' + str(round(accuracy_score(Y_test, y_pred1),4)*100) + '%')
print('RBF SVM Prediction Accuracy = ' + str(round(accuracy_score(Y_test, y_pred2),4)*100) + '%')

Linear SVM Prediction Accuracy = 75.0%
RBF SVM Prediction Accuracy = 78.75%


In [6]:
# Use the most accurate model to classify roof_types in the file Roof_Application_Set.csv
# Going to use classifier1

stage1_Y_pred = classifier1.predict(stage1_X)
all_metrics = ['fp_seg_num','fp_perimeter_len','seg_numTOunique_lengths','fp_avg_seg_len_3long','fp_avg_seg_len_3short','ratio_3lngTO3hrt','footprint_area','boundingbox_area','bbTOfp_ratio','bb_rotation','vector','bb_0_180_axis_len','bb_90_270_axis_len','bb_lngTOshrt_ratio']
stage1_all_data = stage1_data[all_metrics]

#creating lists needed for the creation of the dataframe
index = stage1_data['house_id']
stage1_all_data_df = pd.DataFrame(stage1_all_data, columns=all_metrics, index=index)
stage1_Y_pred_df = pd.DataFrame(stage1_Y_pred, columns=['house_type'], index=index)

#merging the two dataframes into one (using concat because it doesn't throw errors like join or merge)
stage1_final_df = pd.concat([stage1_all_data_df, stage1_Y_pred_df], axis=1, sort=False)
#stage1_final_df.head()

#Once these two dataframes are joined, then we want to split them up by 'house_type'
stage1_R1_df = stage1_final_df[(stage1_final_df)['house_type']=='R1']
stage1_R1C_df = stage1_final_df[(stage1_final_df)['house_type']=='R1C']
stage1_R2_df = stage1_final_df[(stage1_final_df)['house_type']=='R2']
stage1_RMC_df = stage1_final_df[(stage1_final_df)['house_type']=='RMC']

#Save these dataframes as .csv files to be used in Stage 2
#RMC_Y_test = pd.DataFrame(RMC_Y_test, columns=['ratio_flat','ratio_0*','ratio_90*','ratio_180*','ratio_270*']).to_csv('RMC_Y_test.csv')

stage1_R1_df.to_csv('stage1_R1.csv')
stage1_R1C_df.to_csv('stage1_R1C.csv')
stage1_R2_df.to_csv('stage1_R2.csv')
stage1_RMC_df.to_csv('stage1_RMC.csv')

